In [7]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'gray'

In [8]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_diagonal_wavelet_full_pastis.ipynb


In [9]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [10]:
from scipy import spatial
from sklearn.decomposition import PCA

In [11]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [12]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [13]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [14]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [15]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,6968.30,0.01,0.09,0.01,0.01,-0.00,0.00
3,0.01,1799.88,-0.01,-0.01,-0.00,0.00,0.00
4,0.09,-0.01,377.73,-0.00,-0.00,-0.00,-0.00
5,0.01,-0.01,-0.00,58.27,-0.00,0.00,-0.00
6,0.01,-0.00,-0.00,-0.00,7.74,0.00,0.00
7,-0.00,0.00,-0.00,0.00,0.00,0.88,0.00
8,0.00,0.00,-0.00,-0.00,0.00,0.00,0.04


In [16]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,0.00000,0.00006,0.00001,0.00003,-0.00002,0.00002
3,0.00000,1.00000,-0.00001,-0.00004,-0.00003,0.00000,0.00001
4,0.00006,-0.00001,1.00000,-0.00003,-0.00002,-0.00002,-0.00000
5,0.00001,-0.00004,-0.00003,1.00000,-0.00002,0.00000,-0.00001
6,0.00003,-0.00003,-0.00002,-0.00002,1.00000,0.00002,0.00001
7,-0.00002,0.00000,-0.00002,0.00000,0.00002,1.00000,0.00004
8,0.00002,0.00001,-0.00000,-0.00001,0.00001,0.00004,1.00000


In [17]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.1317334524194336

In [18]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.95101443e+06 4.49915171e+05 1.90174264e+04 4.24551919e+02
 6.67184351e+00 6.43894140e-02 3.17720501e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998903,-0.045949,-0.008911,-0.001371,-0.000181,-0.000021,-9.958908e-07
1,0.045502,0.997962,-0.044247,-0.006479,-0.000861,-0.000097,-4.912569e-06
2,0.010850,0.043500,0.998189,-0.039763,-0.005132,-0.000583,-2.956335e-05
3,0.002082,0.008082,0.039152,0.998162,-0.045215,-0.005014,-2.544375e-04
4,0.000367,0.001429,0.006804,0.044608,0.997319,-0.057516,-2.860735e-03
5,0.000063,0.000244,0.001163,0.007530,0.057009,0.997047,-5.088701e-02
6,0.000006,0.000025,0.000118,0.000765,0.005750,0.050637,9.987003e-01


In [19]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00109692, 0.00203756, 0.00181051, 0.00183773, 0.00268089,
       0.00295318, 0.00129971])